In [ ]:
!pip install transformers
!pip install faiss-cpu
!pip install faiss-gpu
!pip install -U bitsandbytes
!pip install qwen_vl_utils
!pip install pandas
!pip install  torchvision
!pip install accelerate
!pip install chromadb
!pip install sentence_transformers
!pip install rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 106.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 

In [ ]:
import torch
import sklearn
from torch import nn
from torchvision import transforms
from PIL import Image

In [ ]:
import re
def preprocess_text(result):
  matches = re.findall(r'\b(yes|no)\b', result, flags=re.IGNORECASE)
  final_answer = matches[-1].lower() if matches else "None"
  return final_answer
def answer_to_number(result):
  if result == "yes":
    return 1
  elif result == "no":
    return 0
def computation(labels,results):
  FN,TN,FP,TP,accur = 0,0,0,0,0
  for i in range(len(labels)):
     if labels[i] == 1 and results[i] == 1:
       TP += 1
     elif labels[i] == 1 and results[i] == 0:
       FN += 1
     elif labels[i] == 0 and results[i] == 1:
       FP += 1
     else:
       TN += 1
  for i in range(len(labels)):
    if labels[i] == results[i]:
      accur += 1
  accuracy = accur/len(labels)
  LR_PLUS = (TP/(TP+FN))/(FP/(FP+TN))
  LR_MINUS = (FN/(TP+FN))/(TN/(FP+TN))
  NPV = TN/(TN+FN)
  answer = {
      "LR+":LR_PLUS,
      "LR-":LR_MINUS,
      "NPV":NPV,
      "accuracy":accuracy
  }
  return answer

In [ ]:
from transformers import BitsAndBytesConfig,AutoModelForImageTextToText,AutoProcessor
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
model_intern = AutoModelForImageTextToText.from_pretrained(
    "OpenGVLab/InternVL3_5-8B-HF",
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
processor_intern = AutoProcessor.from_pretrained(
    "OpenGVLab/InternVL3_5-8B-HF",
    trust_remote_code=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

video_preprocessor_config.json: 0.00B [00:00, ?B/s]

In [ ]:
import pandas as pd
df1 = pd.read_csv('train_offense_facts.csv', on_bad_lines='skip')
df2 = pd.read_csv('test_preprocessed.csv', on_bad_lines='skip')
df1 = df1[['id','label','only_facts']]
df2 = df2[['id','label','facts_and_arguments']]
argument_keywords = [
    'hence',
    'oppose',
    'opposes',
    'opposed',
    'opposing',
    'support',
    'supports',
    'supported',
    'supporting',
    'bailable',
    'granted',
    'rejected'
]

only_facts = []
for fact_arg in df2['facts_and_arguments']:
    sents = fact_arg.split('. ')
    new_sents = []
    for s in sents:
        flag = True
        for key in argument_keywords:
            if key in s:
                flag = False
                break
        if flag:
          new_sents.append(s)
    only_facts.append('. '.join(new_sents))
df2.loc[:, 'only_facts'] = only_facts
test_img_path = f"/content/grey.jpg"
image = Image.open(test_img_path)
image = image.resize((224, 224), Image.BICUBIC)
image = image.convert("RGB")

In [ ]:
results = []
labels = [df2['label'].iloc[i] for i in range(len(df2))]
for i in range(len(df2)):
 text = df2['only_facts'].iloc[i]
 label = df2['label'].iloc[i]

 system_prompt = '''Instruction: You are provided with the facts of a criminal case report.
                    You need to understand how a bail decision is made from the facts in order to respond to the question.'''
 user_prompt = f'''Question: The facts of the current case are the following: {text}
                  You are also given an image of the accused person which is the photo of the accused which you need to see.
                  Should the accused person be granted bail? Please answer in yes/no only.
                  \n Answer:'''

 conversation = [
     {
         "role": "system",
         "content": system_prompt
     },
    {
        "role": "user",
        "content": [
            {"type": "image", "image" : image},
            {"type": "text", "text": user_prompt}
        ]
    }
]

 prompt = processor_intern.apply_chat_template(conversation, add_generation_prompt=True)
 inputs = processor_intern(images=image, text=prompt, return_tensors="pt")
 inputs = inputs.to("cuda")
 generated_output = model_intern.generate(**inputs, return_dict_in_generate=True,
                                         output_scores=True,
                                         do_sample=True,
                                         max_new_tokens=256,
                                         temperature=0.1)

 # Extracting the generated text from the output of the model
 answer_text = processor_intern.decode(generated_output.sequences[0], skip_special_tokens=True)

 # The original prompt includes the "Answer:" prefix, so we need to remove it from the generated text
 # Find the position of the last "Answer:" and take the substring after it.
 answer_start_index = answer_text.rfind("Answer:")
 if answer_start_index != -1:
     answer_text = answer_text[answer_start_index + len("Answer:"):].strip()
 else:
     answer_text = answer_text.strip()

 print(i+1)

 answer_text = preprocess_text(answer_text)
 print(answer_text)
 results.append(answer_to_number(answer_text))

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


1
assistant
Yes


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


2
assistant
No


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...

3313
assistant
No


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


3314
assistant
No


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


3315
assistant
Yes
3316
assistant
Yes


In [ ]:
print("Without RAG:")
print(labels)
print(results)
print(computation(labels,results))

Without RAG:
[np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0)

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np
client = chromadb.Client()
collection = client.create_collection(name="docs", get_or_create=True)

embedder = SentenceTransformer("all-MiniLM-L6-v2").cuda()
docs = [
    df1['only_facts'].iloc[i]  for i in range(len(df1))
]

embeddings = embedder.encode(docs).tolist()

# Split data into smaller batches to avoid exceeding ChromaDB's batch size limit
batch_size = 5000 # Using 5000, which is less than the max_batch_size of 5461
for i in range(0, len(docs), batch_size):
    batch_docs = docs[i:i + batch_size]
    batch_embeddings = embeddings[i:i + batch_size]
    batch_ids = [f"{j}" for j in range(i, min(i + batch_size, len(docs)))]

    collection.add(
        documents=batch_docs,
        embeddings=batch_embeddings,
        ids=batch_ids
    )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def create_message(index,combined_results,combined_labels):
  return f'''case{index+1}:{combined_results[index]}\n
             Based on the facts the final decision of the case is that the bail was {'granted' if combined_labels[index]==1 else 'rejected'}.'''

In [ ]:
results = []
labels = [df2['label'].iloc[i] for i in range(len(df2))]
for i in range(len(df2)):
  results_chroma = collection.query(
    query_embeddings=embedder.encode(df2['only_facts'].iloc[i]).tolist(),
    n_results=3,
    include=["documents", "distances", "metadatas"]
)

  documents = results_chroma['documents'][0]
  sim_score = results_chroma['distances'][0]
  index=results_chroma['ids'][0]
  index = [int(index[i]) for i in range(len(index))]

  combined_labels = [df1['label'].iloc[i] for i in index]

  text = df2['only_facts'].iloc[i]
  label = df2['label'].iloc[i]
  user_prompt = '''You are a bail decision assistant. Study these example cases and apply the pattern to the new case.

EXAMPLE CASES (learn from these):

'''

# Add examples in CLEAR pattern
  for j in range(len(documents)):
    user_prompt += f'''Case {j+1}:
Facts: {documents[j]}
Decision: {"GRANT BAIL (yes)" if combined_labels[j] == 1 else "DENY BAIL (no)"}

'''

  user_prompt += f'''NEW CASE TO DECIDE:
Facts: {text}
Image: image of the accused

INSTRUCTIONS:
1. Compare this new case with the 3 example cases above
2. Look for SIMILAR factors between cases
3. If the new case is similar to majority of the example cases where bail was GRANTED, answer "yes"
4. If the new case is similar to majority of the example cases where bail was DENIED, answer "no"

Important: You MUST base your decision on the comparison with examples.

Decision (yes/no only): '''
  conversation = [

    {
        "role": "user",
        "content": [
            {"type": "image", "image" : image},
            {"type": "text", "text": user_prompt}
        ]
    }
]
  prompt = processor_intern.apply_chat_template(conversation, add_generation_prompt=True)
  inputs = processor_intern(images=image, text=prompt, return_tensors="pt")
  inputs = inputs.to("cuda")
  generated_ids = model_intern.generate(**inputs, return_dict_in_generate=True,
                                         output_scores=True,
                                         do_sample=True,
                                         max_new_tokens=256,
                                         temperature=0.1)
  answer_text = processor_intern.decode(generated_ids.sequences[0], skip_special_tokens=True)
  answer_start_index = answer_text.rfind("Answer:")
  if answer_start_index != -1:
     answer_text = answer_text[answer_start_index + len("Answer:"):].strip()
  else:
     answer_text = answer_text.strip()

  print(i+1)

  answer_text = preprocess_text(answer_text)
  print(answer_text)
  results.append(answer_to_number(answer_text))

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


1
yes


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


2
no


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...

3313
no


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


3314
no


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


3315
yes
3316
yes


In [ ]:
print("With RAG:")
print(labels)
print(results)
print(computation(labels,results))

With RAG:
[np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), n